In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

quora = load_dataset("quora", split="train[:500]")



In [ ]:
quora

Dataset({
    features: ['questions', 'is_duplicate'],
    num_rows: 500
})

In [ ]:
quora = quora.train_test_split(test_size=0.2)

In [ ]:
quora["train"][0]

{'questions': {'id': [703, 704],
  'text': ['How do I send message from one Android phone to another Android phone through bluetooth?',
   'How do I transfer my contacts from one Android phone to another Android phone?']},
 'is_duplicate': False}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
quora=quora.flatten()

In [ ]:
quora["train"][0]

{'questions.id': [703, 704],
 'questions.text': ['How do I send message from one Android phone to another Android phone through bluetooth?',
  'How do I transfer my contacts from one Android phone to another Android phone?'],
 'is_duplicate': False}

In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["questions.text"]])

In [ ]:
tokenized_quora = quora.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=quora["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [ ]:
lm_dataset = tokenized_quora.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained("distilroberta-base")

All PyTorch model weights were used when initializing TFRobertaForMaskedLM.

All the weights of TFRobertaForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    lm_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    lm_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="my_awesome_quora_mlm_model",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/ilyesrezgui/my_awesome_quora_mlm_model into local empty directory.


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=[callback])

Epoch 1/3
5/5 [==============================] - 36s 4s/step - loss: 2.1299 - val_loss: 1.7090
Epoch 2/3
5/5 [==============================] - 2s 453ms/step - loss: 1.8915 - val_loss: 1.9481
Epoch 3/3
5/5 [==============================] - 2s 423ms/step - loss: 1.7252 - val_loss: 1.8581


In [ ]:
text = "UBIAI is a <mask> company."

In [ ]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", "my_awesome_quora_mlm_model")
mask_filler(text, top_k=3)

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at my_awesome_quora_mlm_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


[{'score': 0.21696670353412628,
  'token': 940,
  'token_str': ' private',
  'sequence': 'UBIAI is a private company.'},
 {'score': 0.12906642258167267,
  'token': 285,
  'token_str': ' public',
  'sequence': 'UBIAI is a public company.'},
 {'score': 0.08642949163913727,
  'token': 3382,
  'token_str': ' registered',
  'sequence': 'UBIAI is a registered company.'}]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_quora_mlm_model")
inputs = tokenizer(text, return_tensors="tf")
mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]

In [ ]:
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained("my_awesome_quora_mlm_model")
logits = model(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at my_awesome_quora_mlm_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [ ]:
top_3_tokens = tf.math.top_k(mask_token_logits, 3).indices.numpy()

for token in top_3_tokens:
    print(text.replace(tokenizer.mask_token, tokenizer.decode([token])))

UBIAI is a  private company.
UBIAI is a  public company.
UBIAI is a  registered company.
